### <입실론 소프트 정책에 대한 활성 정책 최초 접촉 MC 제어>

수도코드 출처 : 단단한 강화학습 챕터 5 

알고리즘 파라미터 : 작은 양의 값 $\epsilon$ >0 

초기화 : 
- $\pi$ <- 임의의 입실론 소프트 정책 
- 모든 s $\in S$ , $a \in A(s)$에 대한 (임의의) $Q(s,a) \in R$ 
- Returns(s,a) <- 모든 $s \in S, a \in A(s)$에 대해 빈 리스트 


(각 에피소드에 대해) 무한히 반복 : 
- $\pi$를 따르는 에피소드를 생성 : $S_0, A_0, R_1, ..., S_{T-1}, A_{T-1}, R_T$
- G <- 0 
- 에피소드의 각 단계에 대한 루트, t = T-1, T-2, ..., 0 
 - G <- $\gamma G + R_{t+1}$
 - $S_t, A_t$ 쌍이 $S_0, A_0, S_1, A_1, ..., S_{t-1}, A_{t-1}$ 중에 나타나지 않는다면 : 
 
 > G를 $Returns(S_t, A_t)$ 리스트에 추가 
 
 > $Q(S_t, A_t)$ <- $average(Returns(S_t, A_t))$ 
 
 > $A^*$ <- $argmax_a Q(S_t,a)$ (최대로 만드는 a의 값이 2개 이상이면 그 중 하나를 임의로 선택한다.) 
 
 > 모든 $ a \in A(S_t)$에 대해 
 
  > $\pi(a|S_t)$ <-
  >> 1) $1- \epsilon + \epsilon/|A(S_t)|$ a = A^* 인 경우 
  
  >> 2) $\epsilon / |A(S_t)|$   a != A^* 인 경우




**구현해야 하는 것** 
- Q(s,a) : input 으로 s,a 를 받아 가치 값을 반환하는 함수 

-  $A^*$ <- $argmax_a Q(S_t,a)$ 에서 같은 같이 나올 경우 임의로 둘 중 하나 선택하는 함수 

- returns(s,a) : 모든 s, a에 대해 빈 리스트라고 함. 

- $\pi(a,s)$ : class 내 내부 함수로 넣어서 갱신도리 수 있도록 해야함. 변경 되도록 만들려면 dic 형태가 좋으려나? 


**필요한 것** 
- $\epsilon$ : init 에서 부여할 것 

- reward func

**함수/데이터의 형태** 
- class epsilon_soft_MC : 

> def __init__(self, S, A, reward_func, gamma = 0.9, num_episode = 10, len_episode = 20) 

> def choice_sample(self, list, prob) : 기존 함수 유지 

> def make_episode(self, start_s, T, pi) : 기존 함수 유지  

> def update_returns(self) : 행동 가치 함수 Q(s,a)에 맞게 수정할 것. 

**전제**
- 상태 s에서 행동 a를 했을 때 결정론적으로 s' 상태로 변화한다. 

<외부 함수> 
- def reward_func(s',a,s) : 

- def choose_random_max(lst) : 최대값이 2개 이상인 경우, 임의로 1개의 최대값을 만들어낸 행동 a를 산출 


<고민점>
- 정책 $\pi$를 어떤 데이터 형식으로 표현하여 값을 갱신할 것인가? 
> dict가 값을 불러오기에 가장 깔끔할 듯 

- returns(s,a) 를 어떤 데이터 형식으로 표현할 것인가? 
> list라고 정의되어 있으니, 이중 리스트로 해서 returns[s][a] 형태로 값을 찾자. 

In [2]:
# 테스트 용 임시 데이터 
S = list(range(100)) 
A = list(range(-5,5))

In [22]:
from collections import defaultdict
import random
import numpy as np

In [60]:
# 외부용 함수 reward_func 간략 구현 (이전 예시 활용)
def reward_func(next_s, a, s) : 
    # next_s 와 s의 차이가 짝수이면 +1, 홀수면 -1 
    # 단, a의 크기에 반비례함. 
    if abs(next_s - s) %2 == 0 : reward = 1 
    else : reward = -1
    
    if a == 0 : 
        return 0 
    else : 
        return reward / a # 즉, a가 양수이며 짝수이며, 가능한 작을 때 (=2) 일 때 최대의 보상이 주어지도록 설정 
    
#최대값이 2개 이상인 경우, 임의로 1개의 최대값을 만들어낸 행동 a를 산출 
def choose_random_max(lst) :
    max_arg = np.where(np.array(lst) >= max(lst))
    return random.choice(list(max_arg)[0]) #max_arg가 array 형태로 안에 있는 list를 꺼내기 위해 [0] 사용 
    
    
    

In [65]:
a = [1,2,3,5,4,5]
print(choose_random_max(a))


3


In [11]:
# 최초 접촉 MC 예측 알고리즘 참고 
# pi(a,s) 를 내부 dic 형태로 구현

class epsilon_soft_MC : 
    def __init__(self, S, A, reward_func, gamma = 0.9, num_episode = 10, len_episode = 20) : 
        self.S = S 
        self.A = A 
        self.reward_func = reward_func
        self.pi = self.initiate_pi()
        self.gamma = 0.9 
        self.num_episode = num_episode
        self.T = len_episode
        
        self.set_episode = [] # 추후에 어떤 episode들이 있었나 확인용 
        
        self.V, self.return_lst, self.s_prob = self.initiate() 
    
    
    def initiate_pi(self) : # pi를 dict로 구성. 이를 초기화하는 함수
        pi_dic = defaultdict(float)
        for s in self.S :
            for a in self.A :  
                pi_dic[(a,s)] = 1/len(self.A)
        return pi_dic
    
    def initiate(self) : 
        V = defaultdict(float)
        s_prob = defaultdict(list)
        for s in self.S : 
            V[s] = 0
            s_prob[s] = [0]*len(self.A)
            for index, a in enumerate(self.A) : 
                s_prob[s][index] = self.pi[(a,s)]

        return_lst = [0]*len(self.S)
        return V, return_lst, s_prob
    
    def choice_sample(self, s) : 
        a= random.choices(self.A, weights = self.s_prob[s])
        
        # 수정. a[0] +s 가 s의 범위 안에 들어오도록 수정 
        return a[0], min(max(s+a[0],0), 99)  
    
    def make_episode(self, start_s, T) :
        s = start_s
        episode = {"S" : [], "A" : [], "R" : []}
        episode["R"].append(0) # R_0 값 부여 
        for _ in range(T) : 
            episode["S"].append(s)
            a, next_s = self.choice_sample(s)
            r = self.reward_func(next_s, a, s)
            episode["A"].append(a)
            episode["R"].append(r)
            s = next_s
        return episode
 
"""
# 최초 접촉 버전의 update_returns 함수임. 
    def update_returns(self) :
        for _ in range(self.num_episode) :
            # 시작 s는 시작 가정에 따라 랜덤하게 산출하겠음. 
            start_s = random.choice(self.S)
            set_s = set() # t 단계까지 나왔던 s 확인 
            set_s.add(start_s)
            episode = self.make_episode(start_s, self.T)
            G = 0 
            G_lst = [0]*self.T
            
            # G와 returns(S_t)를 업데이트 하는 순서가 각각 역순이라 따로 구현
            for t in reversed(range(self.T)) : 
                G = self.gamma *G + episode["R"][t]
                G_lst[t] = G
                
            for t in range(self.T): 
                s = episode["S"][t]
                if s not in set_s : 
                    self.return_lst[s] = G_lst[self.T-t-1]
                    self.V[s] = sum(self.return_lst) / len([i for i in self.return_lst if i != 0])
                set_s.add(s)         
"""        
        
                
        

'\n# 최초 접촉 버전의 update_returns 함수임. \n    def update_returns(self) :\n        for _ in range(self.num_episode) :\n            # 시작 s는 시작 가정에 따라 랜덤하게 산출하겠음. \n            start_s = random.choice(self.S)\n            set_s = set() # t 단계까지 나왔던 s 확인 \n            set_s.add(start_s)\n            episode = self.make_episode(start_s, self.T)\n            G = 0 \n            G_lst = [0]*self.T\n            \n            # G와 returns(S_t)를 업데이트 하는 순서가 각각 역순이라 따로 구현\n            for t in reversed(range(self.T)) : \n                G = self.gamma *G + episode["R"][t]\n                G_lst[t] = G\n                \n            for t in range(self.T): \n                s = episode["S"][t]\n                if s not in set_s : \n                    self.return_lst[s] = G_lst[self.T-t-1]\n                    self.V[s] = sum(self.return_lst) / len([i for i in self.return_lst if i != 0])\n                set_s.add(s)         \n'

In [12]:
# 기존 활용 함수들 점검 
test = epsilon_soft_MC(S,A,reward_func) 
print(test.pi)

defaultdict(<class 'float'>, {(-5, 0): 0.1, (-4, 0): 0.1, (-3, 0): 0.1, (-2, 0): 0.1, (-1, 0): 0.1, (0, 0): 0.1, (1, 0): 0.1, (2, 0): 0.1, (3, 0): 0.1, (4, 0): 0.1, (-5, 1): 0.1, (-4, 1): 0.1, (-3, 1): 0.1, (-2, 1): 0.1, (-1, 1): 0.1, (0, 1): 0.1, (1, 1): 0.1, (2, 1): 0.1, (3, 1): 0.1, (4, 1): 0.1, (-5, 2): 0.1, (-4, 2): 0.1, (-3, 2): 0.1, (-2, 2): 0.1, (-1, 2): 0.1, (0, 2): 0.1, (1, 2): 0.1, (2, 2): 0.1, (3, 2): 0.1, (4, 2): 0.1, (-5, 3): 0.1, (-4, 3): 0.1, (-3, 3): 0.1, (-2, 3): 0.1, (-1, 3): 0.1, (0, 3): 0.1, (1, 3): 0.1, (2, 3): 0.1, (3, 3): 0.1, (4, 3): 0.1, (-5, 4): 0.1, (-4, 4): 0.1, (-3, 4): 0.1, (-2, 4): 0.1, (-1, 4): 0.1, (0, 4): 0.1, (1, 4): 0.1, (2, 4): 0.1, (3, 4): 0.1, (4, 4): 0.1, (-5, 5): 0.1, (-4, 5): 0.1, (-3, 5): 0.1, (-2, 5): 0.1, (-1, 5): 0.1, (0, 5): 0.1, (1, 5): 0.1, (2, 5): 0.1, (3, 5): 0.1, (4, 5): 0.1, (-5, 6): 0.1, (-4, 6): 0.1, (-3, 6): 0.1, (-2, 6): 0.1, (-1, 6): 0.1, (0, 6): 0.1, (1, 6): 0.1, (2, 6): 0.1, (3, 6): 0.1, (4, 6): 0.1, (-5, 7): 0.1, (-4, 7): 0.

In [71]:
# 최초 접촉 MC 예측 알고리즘 참고 
# pi(a,s) 를 내부 dic 형태로 구현

class epsilon_soft_MC : # epsilon 추가 
    def __init__(self, S, A, reward_func, epsilon = 0.001, gamma = 0.9, num_episode = 10, len_episode = 20) : 
        self.S = S 
        self.A = A 
        self.epsilon = epsilon 
        self.reward_func = reward_func
        self.pi = self.initiate_pi()
        self.gamma = 0.9 
        self.num_episode = num_episode
        self.T = len_episode
        
        self.set_episode = [] # 추후에 어떤 episode들이 있었나 확인용 
        
        self.Q, self.return_lst, self.s_prob = self.initiate() 
    
    
    def initiate_pi(self) : # pi를 dict로 구성. 이를 초기화하는 함수
        pi_dic = defaultdict(float)
        for s in self.S :
            for a in self.A :  
                pi_dic[(a,s)] = 1/len(self.A)
        return pi_dic
    
    def initiate(self) : # V 대신 Q(a,s)를 초기화해야 함. 
        Q = defaultdict(float)
        s_prob = defaultdict(list)
        for s in self.S : 
            s_prob[s] = [0]*len(self.A)
            for index, a in enumerate(self.A) : 
                Q[(a,s)] = 0 
                s_prob[s][index] = self.pi[(a,s)]

        return_lst = [[0]*len(self.A)]*len(self.S) # return_lst의 데이터 형식 변경
        return Q, return_lst, s_prob
    
    def choice_sample(self, s) : 
        a= random.choices(self.A, weights = self.s_prob[s])
        
        # 수정. a[0] +s 가 s의 범위 안에 들어오도록 수정 
        return a[0], min(max(s+a[0],0), 99)  
    
    def make_episode(self, start_s, T) :
        s = start_s
        episode = {"S" : [], "A" : [], "R" : []}
        episode["R"].append(0) # R_0 값 부여 
        for _ in range(T) : 
            episode["S"].append(s)
            a, next_s = self.choice_sample(s)
            r = self.reward_func(next_s, a, s)
            episode["A"].append(a)
            episode["R"].append(r)
            s = next_s
        return episode 
 
    def update_returns(self) : # epsilon - soft 행동 가치 함수에 맞춰 변경
        for _ in range(self.num_episode) :
            # 시작 s는 시작 가정에 따라 랜덤하게 산출하겠음. 
            start_s = random.choice(self.S)
            set_s = set() # t 단계까지 나왔던 s 확인 
            set_s.add(start_s)
            episode = self.make_episode(start_s, self.T)
            G = 0 
            G_lst = [0]*self.T
            
            # G와 returns(S_t)를 업데이트 하는 순서가 각각 역순이라 따로 구현
            for t in reversed(range(self.T)) : 
                G = self.gamma *G + episode["R"][t]
                G_lst[t] = G
                
            for t in range(self.T): 
                s = episode["S"][t]
                a = episode["A"][t]
                if s not in set_s : 
                    self.return_lst[s][a] = G_lst[self.T-t-1]
                    
                    # V 대신 Q로 변경 필요 
                    length = 0 
                    for _ in self.S : length += len([a for a in self.return_lst[_] if a != 0])
                    self.Q[(a,s)] = sum([sum(s) for s in self.return_lst]) / length
                    for s in self.S : 
                        Q_lst = [] 
                        for a in self.A : Q_lst.append(self.Q[(a,s)])
                        best_A = choose_random_max(Q_lst)
                        for a in self.A : 
                            if a == best_A : self.pi[(a,s)] = 1 - self.epsilon + self.epsilon/len(self.A)
                            else : self.pi[(a,s)] = self.epsilon / len(self.A)
                        
                set_s.add(s)         
       
        
                
        

In [73]:
# 기존 활용 함수들 점검 
test = epsilon_soft_MC(S,A,reward_func) 
test.update_returns() 
print(test.Q)
#print(test.pi)
#print(test.return_lst)


defaultdict(<class 'float'>, {(-5, 0): 0, (-4, 0): 0, (-3, 0): -1.073721556801192, (-2, 0): 0, (-1, 0): 0, (0, 0): -1.1544154659924513, (1, 0): -0.995672459280584, (2, 0): 0, (3, 0): 0, (4, 0): 0, (-5, 1): 0, (-4, 1): -1.0549015965025546, (-3, 1): 0, (-2, 1): 0, (-1, 1): 0, (0, 1): 0, (1, 1): -1.0023059375756473, (2, 1): 0, (3, 1): 0, (4, 1): 0, (-5, 2): -0.12526461244974627, (-4, 2): 0, (-3, 2): 0, (-2, 2): 0, (-1, 2): 0, (0, 2): 0, (1, 2): 0, (2, 2): 0, (3, 2): -1.1898953096507987, (4, 2): 0, (-5, 3): 0, (-4, 3): 0, (-3, 3): 0, (-2, 3): 0, (-1, 3): 0, (0, 3): 0, (1, 3): 0, (2, 3): 0, (3, 3): 0, (4, 3): 0, (-5, 4): 0, (-4, 4): 0, (-3, 4): 0, (-2, 4): -0.09252224878307953, (-1, 4): 0, (0, 4): 0, (1, 4): 0, (2, 4): 0, (3, 4): 0, (4, 4): 0, (-5, 5): 0, (-4, 5): 0, (-3, 5): 0, (-2, 5): 0, (-1, 5): -1.4097637857670031, (0, 5): 0, (1, 5): 0, (2, 5): 0, (3, 5): 0, (4, 5): 0, (-5, 6): 0, (-4, 6): 0, (-3, 6): 0, (-2, 6): -0.14022295211641292, (-1, 6): 0, (0, 6): 0.34725813121692073, (1, 6): 0,

추후 수정점 : pi(a,s), Q(a,s) 를 전부 pi(s,a), Q(s,a)로 변경할 것. 가독성이 안 좋다. 